## Load Necessary Library

In [1]:
from sklearn.metrics import accuracy_score,mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import make_pipeline,make_union
from sklearn.preprocessing import MinMaxScaler, Imputer,PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV,KFold,train_test_split
import warnings
from sklearn.feature_selection import SelectPercentile,SelectKBest, VarianceThreshold
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#Need to run all Data Prep Notebook cells
%store -r data

In [4]:
data.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,160,43,23,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,False,0,22,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,145,20,51,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,False,15,50,100.0,2421.0,30000.00
1000004038,Where is Hank?,145,20,51,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,False,3,50,220.0,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,142,35,51,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,False,1,50,1.0,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,107,20,51,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,False,14,50,1283.0,1283.0,19500.00


## Make a new feature

In [5]:
data["duration"] = ((data["deadline"]  - data['launched']) / np.timedelta64(1, 'D')).astype(int)

In [30]:
data["month_launched"] = data["launched"].dt.month
data["month_deadline"] = data["deadline"].dt.month
data["launch_day_of_week"] = data["launched"].dt.dayofweek

#projects = data['launched'].groupby([data['launched'].dt.year.rename('year'), data['launched'].dt.month.rename('month')]).agg({'count'})



In [7]:
def state_to_num(x):
    if x < 0.5: return 0
    elif 0.5<= x < 1 : return 1
    elif 1<= x <= 1.5: return 2
    else: return 3
data["state_num"] = [state_to_num(x)for x in  (data.usd_pledged_real/data.usd_goal_real)]

In [8]:
data.columns

Index(['name', 'category', 'main_category', 'currency', 'deadline', 'goal',
       'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged',
       'usd_pledged_real', 'usd_goal_real', 'duration', 'month_launched',
       'month_deadline', 'launch_day_of_week', 'state_num'],
      dtype='object')

In [9]:
feature = data[['category', 'main_category', 'currency', 'goal','duration', 'country', 'month_launched', 'month_deadline', 'launch_day_of_week']]
outcome = data[['state_num']]
feature_name = data[["name"]]

In [10]:
feature.head()

,category,main_category,currency,goal,duration,country,month_launched,month_deadline,launch_day_of_week
ID,,,,,,,,,
1000002330,160,43,23,1000.0,58,22,8,10,1
1000003930,145,20,51,30000.0,59,50,9,11,5
1000004038,145,20,51,45000.0,44,50,1,2,5
1000007540,142,35,51,5000.0,29,50,3,4,5
1000011046,107,20,51,19500.0,55,50,7,8,5


## Split test and train data set

In [11]:
train_features_small, validation_features, train_outcome_small, validation_outcome = train_test_split(
    feature,
    outcome,
    test_size = 0.3
)
train_name_small, validation_name_features, train_outcome_small_name, validation_name_outcome = train_test_split(
    feature_name,
    outcome,
    test_size = 0.3
)

# Predict the State by Its Name

In [69]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

In [35]:
text_clf = make_pipeline(
                    CountVectorizer(),
                    TfidfTransformer(),
                    SGDClassifier(penalty='l2', tol=None,max_iter=10)
)

param = {'countvectorizer__ngram_range': [(1, 1), (1, 2)],
    'tfidftransformer__use_idf': (True, False),
    "sgdclassifier__alpha":[1e-3,0.001],
}

grid_search_text = GridSearchCV(text_clf, n_jobs=-1,cv=5,iid=False,param_grid=param)
grid_search_text.fit(train_name_small.name,train_outcome_small_name)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'countvectorizer__ngram_range': [(1, 1), (1, 2)], 'tfidftransformer__use_idf': (True, False), 'sgdclassifier__alpha': [0.001, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [36]:
print(classification_report(validation_outcome, grid_search_text.best_estimator_.predict(validation_name_features.name)))

             precision    recall  f1-score   support

      False       0.64      0.99      0.78     72446
       True       0.35      0.01      0.02     40313

avg / total       0.54      0.64      0.51    112759



In [37]:
text_clf = make_pipeline(
                    CountVectorizer(),
                    TfidfTransformer(),
                    SelectKBest(k=10),
                    MLPClassifier(hidden_layer_sizes=(500,10,2),  
                                  activation='relu', 
                                   batch_size='auto', 
                                  learning_rate_init=0.01, power_t=0.5, 
                                  max_iter=1000, shuffle=True,
                                  tol=0.0001, verbose=False, warm_start=False,
                                  momentum=0.9, nesterovs_momentum=True,
                                    early_stopping=False, validation_fraction=0.1,
                                  beta_1=0.9,beta_2=0.999, epsilon=1e-08)
                                )


param = {'countvectorizer__ngram_range': [(1, 1), (1, 2)],
    'tfidftransformer__use_idf': (True, False)
}

grid_search_text = GridSearchCV(text_clf, n_jobs=-1,cv=5,iid=False,param_grid=param)
grid_search_text.fit(train_name_small.name,train_outcome_small_name)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'countvectorizer__ngram_range': [(1, 1), (1, 2)], 'tfidftransformer__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [38]:
print(classification_report(validation_outcome, grid_search_text.predict(validation_name_features.name)))


             precision    recall  f1-score   support

      False       0.64      1.00      0.78     72446
       True       0.00      0.00      0.00     40313

avg / total       0.41      0.64      0.50    112759



In [39]:
grid_search_text.best_score_

0.6514255352648519

In [40]:
grid_search_text.score(validation_name_features.name,validation_name_outcome)

0.6434874378098422

In [41]:
name_proba = grid_search_text.predict_log_proba(validation_features)

# Predict the State by other value

In [12]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_features="auto",max_depth=4, min_samples_split=2)
pipe = make_pipeline(VarianceThreshold(.1),
                     SelectKBest(k=9),
                     rfc
                     )

params ={'randomforestclassifier__n_estimators':range(20,50,10),
        }

grid_search_rf = GridSearchCV(pipe, params,
                               cv = KFold(n_splits=10, shuffle=True),n_jobs = -1)

In [13]:
grid_search_rf.fit(train_features_small,train_outcome_small)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.1)), ('selectkbest', SelectKBest(k=9, score_func=<function f_classif at 0x0000028CDE3B88C8>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'randomforestclassifier__n_estimators': range(20, 50, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
print(classification_report(validation_outcome, grid_search_rf.best_estimator_.predict(validation_features)))

             precision    recall  f1-score   support

          0       0.62      0.99      0.76     69085
          1       0.00      0.00      0.00      2557
          2       0.64      0.05      0.09     29360
          3       0.00      0.00      0.00     11757

avg / total       0.55      0.62      0.49    112759



In [15]:
grid_search_rf.best_params_

{'randomforestclassifier__n_estimators': 40}

In [16]:
grid_search_rf.score(validation_features,validation_outcome)

0.6197820129657057

In [17]:
rf_proba = grid_search_rf.predict_log_proba(validation_features)
len(name_proba)
len(rf_proba)

NameError: name 'name_proba' is not defined

# Combine two Prediction Together

In [42]:
total_proba = name_proba * rf_proba

ValueError: operands could not be broadcast together with shapes (9,2) (112759,2) 

In [108]:
total_proba

array([[0.27718412, 0.79689983],
       [0.52655006, 0.43774873],
       [0.29568657, 0.75431854],
       ...,
       [0.10719979, 1.48391614],
       [0.657058  , 0.30049206],
       [0.4828025 , 0.47806432]])

In [112]:
total_pred = [(x[0] >= x[1])for x in total_proba]

In [113]:
validation_outcome["pred"] = total_pred

In [114]:
len(validation_outcome[validation_outcome["pred"]==validation_outcome["state"]])/len(validation_outcome)

0.643489018663616

### Neural Network test

In [118]:
from sklearn.neural_network import MLPClassifier

    
pipe = make_pipeline(VarianceThreshold(0.1),
                    SelectKBest(k = 9),
                     MLPClassifier(hidden_layer_sizes=(13,5,),  
                                  activation='relu', 
                                   batch_size='auto', 
                                  learning_rate_init=0.01, power_t=0.5, 
                                  max_iter=1000, shuffle=True,
                                  tol=0.0001, verbose=False, warm_start=False,
                                  momentum=0.9, nesterovs_momentum=True,
                                    early_stopping=False, validation_fraction=0.1,
                                  beta_1=0.9,beta_2=0.999, epsilon=1e-08)
                                )
params ={'mlpclassifier__alpha':[0.0001,0.001,0.01],
         'mlpclassifier__solver':['lbfgs',  'adam']
        }

grid_search_nn = GridSearchCV(pipe, param_grid = params,cv = KFold(n_splits=5, shuffle=True),n_jobs=-1)

In [119]:
train_outcome_small.state = train_outcome_small.state.astype(int)

In [120]:
grid_search_nn.fit(train_features_small,train_outcome_small)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.1)), ('selectkbest', SelectKBest(k=9, score_func=<function f_classif at 0x000001A742D0AA60>)), ('mlpclassifier', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'mlpclassifier__alpha': [0.0001, 0.001, 0.01], 'mlpclassifier__solver': ['lbfgs', 'adam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [121]:
grid_search_nn.score(validation_features,validation_outcome)

0.6434342269796646